In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
from time import sleep
import pandas as pd
import numpy as np

In [2]:
driver = webdriver.PhantomJS(executable_path=r'C:\phantomjs-2.1.1-windows\bin\phantomjs.exe')
driver2 = webdriver.PhantomJS(executable_path=r'C:\phantomjs-2.1.1-windows\bin\phantomjs.exe')

In [12]:
driver.get("https://esports-betonline.ultraplay.net/esports/today")
driver2.get("https://esports-betonline.ultraplay.net/esports/program/0")

sleep(5)
html = driver.execute_script("return document.getElementsByTagName('html')[0].innerHTML")
html2 = driver2.execute_script("return document.getElementsByTagName('html')[0].innerHTML")
soup = BeautifulSoup(html, "html.parser")
soup2 = BeautifulSoup(html2, "html.parser")


In [13]:
# get soup
print(soup)

<head><style type="text/css">.ng-animate.item:not(.left):not(.right){-webkit-transition:0s ease-in-out left;transition:0s ease-in-out left}</style><style type="text/css">@charset "UTF-8";[ng\:cloak],[ng-cloak],[data-ng-cloak],[x-ng-cloak],.ng-cloak,.x-ng-cloak,.ng-hide:not(.ng-hide-animate){display:none !important;}ng\:form{display:block;}</style>
<title class="ng-binding ng-scope" data-ng-bind="getTitle()" data-ng-controller="TitleController" data-ng-init="setTitle('betonline')">Today - BetOnline</title>
<meta charset="utf-8"/>
<base href="/"/>
<meta content="!" name="fragment"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<meta content="INDEX, FOLLOW" name="ROBOTS"/>
<meta class="ng-scope" content="Join betonline" data-ng-controller="TitleController" data-ng-init="setMetaDescription('Join betonline')" name="description"/>
<link href="/Content/images/fav-icon.png" rel="shortcut icon" type="image/x-icon"/>
<link href="data:;base64,iVBORw0KGgo=" rel="icon"/>


In [53]:
# get leagues

sportList = soup.find_all('div',{'class': 'panel panel-default panel-sport hidden-lg ng-scope'})

#print(sportList[0])

for sport in sportList:
    #print(sportList)
    sportName = sport.find('div',{'class': 'row panel-heading border-none'}).text.strip()
    
    print(sportName)
    
    leagueList = sport.find_all('div', {'class': 'panel panel-default margin-bottom-none panel-league ng-scope'})
    
    for league in leagueList:
        leagueName = league.find('div', {'class': 'col-xs-10 col-sm-11 text-left panel-heading match-detail-league-name'}) 
        if leagueName is not None:
            print(leagueName.text.strip())

Dota 2
H-Cup Season
Vainglory
Vainglory World Championship


In [6]:
# get 1x2 value
#games = soup.find_all('div', {'class': 'panel-match-detail'})
#print(games[1])

In [70]:
todayOdds = pd.DataFrame(columns=['UUID_O','GameID','TeamID','Moneyline','DatePulled','LookupID_O'])
todayGames = pd.DataFrame(columns=['UUID_G','Sport','League','GameID','TeamA','TeamB','Date','LookupID_G'])
todayTeams = pd.DataFrame(columns=['UUID_T','TeamName','Sport','League','LookupID_T'])

i = 0

sportList = soup.find_all('div',{'class': 'panel panel-default panel-sport visible-lg ng-scope'})
for sport in sportList:
    sportName = sport.find('div',{'class': 'col-xs-6 ng-binding'}).text.strip()

    print(sportName)
    
    leagueList = sport.find_all('div', {'class': 'panel panel-default margin-bottom-none panel-league ng-scope'})
    for league in leagueList:
        leagueName = league.find('div', {'class': 'col-xs-10 col-sm-11 text-left panel-heading match-detail-league-name'}).text.strip() 
        
        if leagueName is not None:
            print(leagueName)        

        gameList = league.find_all('div', {'class': 'panel-match-detail ng-scope'})

        #print(gameList)
        
        for game in gameList:
            teamA = game.find('div', {'class':'match-detail-teamA-row'})
            teamB = game.find('div', {'class':'match-detail-teamB-row'})
            
            if teamA is not None:
                teamName1 = teamA.find('div', {'class': 'match-detail-team-name'}).text.strip()
                teamName2 = teamB.find('div', {'class': 'match-detail-team-name'}).text.strip()
                Odds1 = teamA.find_all('div', {'class': 'col-xs-2 col-sm-1 event-link'})
                Odds2 = teamB.find_all('div', {'class': 'col-xs-2 col-sm-1 event-link'})

                if 'suspended' not in (game.get('class')):
                    ML1 = Odds1[0].text.strip()
                    ML2 = Odds2[0].text.strip()

                # create dataframe to hold entry
                pdOdds = pd.DataFrame([[0, 0, 0, ML1, 0, 0],[0, 0, 0, ML2, 0, 0]],
                                      columns=['UUID_O','GameID','TeamID','Moneyline','DatePulled','LookupID_O'])
                
                pdGames = pd.DataFrame([[0, sportName, leagueName, 0, 0, 0, 0, 0, 0]],
                                      columns=['UUID_G','Sport','League','GameID','TeamA','TeamB','Date','LookupID_G'])
                    
                pdTeams = pd.DataFrame([[0, teamName1, sportName, leagueName, 0], [0, teamName2, sportName, leagueName, 0]],
                                      columns=['UUID_T','TeamName','Sport','League','LookupID_T'])             

                todayOdds = todayTeams.append(pdOdds)                
                todayGames = todayTeams.append(pdGames)
                todayTeams = todayTeams.append(pdTeams)
                i = i+1
print(todayTeams)

Dota 2
H-Cup Season
VGJ.ThunderKeen Gaming.Luminous
Vainglory
Vainglory World Championship
Cloud9Tribe Gaming
  UUID_T     TeamName      Sport                        League LookupID_T
0      0  VGJ.Thunder     Dota 2                  H-Cup Season          0
0      0       Cloud9  Vainglory  Vainglory World Championship          0
